In [1]:
#  Bibliotecas
import time
import pyautogui as po
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import datetime
from selenium.common.exceptions import WebDriverException     


In [3]:
# Xpaths
xpath_login_feds = ' //*[@id="txtUsuario"] '     
xpath_senha_feds = '//*[@id="pwdSenha"] '  
xpath_adv_feds = ' //*[@id="tr0"] '
xpath_menu_consulta_feds = '//*[@id="main-menu"]/li[4]/a' 
xpath_consul_processual_feds ='//*[@id="menu-ul-3"]/li/a'
xpath_carlos_feds ='//*[@id="numNrProcesso"]'

# Outras variaveis 
url_feds1 = 'https://www.jusbrasil.com.br/consulta-processual/'
delay0 = 10

In [4]:
# Parte PD
# Importaçao da planilha com os processos
plan_name = 'Planilha_Processos.xlsx'
sheet_name = 'feds1'

tabela_original = pd.read_excel(plan_name, sheet_name = sheet_name) 

processos_df = pd.DataFrame() 
processos_df["Processo"] = tabela_original["Processo"] 

In [5]:
# Puxar o nav
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service) 
driver.get(url_feds1) 

# Pegar Login e Senha
login = po.prompt(text='Insira seu login: ', title='Login' )
time.sleep(0.5)
senha = po.password(text ='Insira sua senha: ',title='Senha: ', mask='*')

inicio = datetime.now()

# Pop-up accept
time.sleep(3)
driver.switch_to.alert.accept()

# Por login e senha
driver.find_element(By.XPATH,xpath_login_feds).send_keys(login)

# inicio do Fast break; Pegar id das telas 
id_wind1 = driver.window_handles 
# fim do Fast break

driver.find_element(By.XPATH,xpath_senha_feds).send_keys(senha+ Keys.RETURN)

# Login adevogrado 
WebDriverWait(driver, delay0).until(EC.element_to_be_clickable((By.XPATH, xpath_adv_feds))).click()

# indo pra tela de consulta processual
WebDriverWait(driver, delay0).until(EC.element_to_be_clickable((By.XPATH, xpath_menu_consulta_feds))).click()
WebDriverWait(driver, delay0).until(EC.element_to_be_clickable((By.XPATH, xpath_consul_processual_feds))).click()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 108.0.5359
[WDM] - Get LATEST chromedriver version for 108.0.5359 google-chrome
[WDM] - Driver [C:\Users\caiow\.wdm\drivers\chromedriver\win32\108.0.5359.71\chromedriver.exe] found in cache


In [6]:
# Trabalho de corno; corrigir
começo = 0

for i in range (começo,tabela_original.shape[0]):
    
    # tratamento dos dados da planilha
    texto_feds1 = processos_df.loc[i,"Processo"]    
    
    # Now click on carlos
    carlos_feds1 = driver.find_element(By.XPATH, xpath_carlos_feds)
    carlos_feds1.send_keys(texto_feds1)
    carlos_feds1.send_keys(Keys.ENTER) 

    # Pegar data
    xpath_data_feds = ' //*[contains(@id,"trEvento")]/td[2]'
    time.sleep(2)
    
    try:
        data_feds1 = (WebDriverWait(driver,25).until(EC.presence_of_element_located((By.XPATH,xpath_data_feds))).text)   
        print(data_feds1)
        data_feds1 = data_feds1[0:10]
        data_feds1 = datetime.strptime( data_feds1, "%d/%m/%Y")
        processos_df.loc[i,'Data'] = data_feds1 
        
        # A volta
        driver.back()
    except TimeoutException:
        print(f'Processo: {texto_feds1} não achado')
        processos_df.loc[i,'Data'] = 'Não achado '
        carlos_feds1.clear()
    
    i=i+1
    

13/12/2021 13:55:38
27/08/2021 12:20:54
21/12/2022 14:27:47
15/12/2022 15:21:16
17/11/2021 18:37:38
21/06/2022 12:33:33
11/05/2021 15:23:30
31/08/2021 05:27:30
28/07/2022 01:18:04
Processo: 05025164-38.2021.4.02.9445 não achado
07/10/2022 14:59:08
06/05/2022 16:00:23
01/12/2022 17:01:13
25/05/2022 12:42:56
07/12/2022 12:17:08


In [7]:
# finalizar
driver.quit() 

# exportat excel
processos_df.to_excel(" Planilha_Processos_conferido.xlsx", sheet_name = 'Feds1',index = False)
print("Tempo levado: ",datetime.now()-inicio)
#po.alert("Pronto!")

Tempo levado:  0:01:26.736973
